In [1]:
import pandas as pd
import time
import requests
import os
import re
from sqlalchemy import create_engine
# Librerías para web scraping
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

In [2]:
# Configurar opciones de visualización
pd.set_option('display.max_rows', None)  # Mostrar todas las filas
pd.set_option('display.max_columns', None)  # Mostrar todas las columnas
pd.set_option('display.max_colwidth', None)

In [3]:
# Declaramos la variable donde se encuentra nuestro driver
driver_path = r"C:\Users\Jesùs\Downloads\chromedriver.exe"
# Declaramos la variable web a scrapear
webs = ["https://volvorepuestos.com.pe/repuestos-camiones-volvo/fh/cabina.html",
        "https://volvorepuestos.com.pe/repuestos-camiones-volvo/fh/frenos.html",
        "https://volvorepuestos.com.pe/repuestos-camiones-volvo/fh/motor.html",
        "https://volvorepuestos.com.pe/repuestos-camiones-volvo/fh/suspension.html",
        "https://volvorepuestos.com.pe/repuestos-camiones-volvo/fh/transmision.html",
        "https://volvorepuestos.com.pe/repuestos-camiones-volvo/fh/electrico.html",
        "https://volvorepuestos.com.pe/repuestos-camiones-volvo/fh/mantenimiento.html",     
        "https://volvorepuestos.com.pe/repuestos-camiones-volvo/fh/direccion.html",
        "https://volvorepuestos.com.pe/repuestos-camiones-volvo/fm-fmx/cabina.html",
        "https://volvorepuestos.com.pe/repuestos-camiones-volvo/fm-fmx/frenos.html",
        "https://volvorepuestos.com.pe/repuestos-camiones-volvo/fm-fmx/motor.html",
        "https://volvorepuestos.com.pe/repuestos-camiones-volvo/fm-fmx/suspension.html",
        "https://volvorepuestos.com.pe/repuestos-camiones-volvo/fm-fmx/transmision.html",
        "https://volvorepuestos.com.pe/repuestos-camiones-volvo/fm-fmx/electrico.html",
        "https://volvorepuestos.com.pe/repuestos-camiones-volvo/fm-fmx/mantenimiento.html",
        "https://volvorepuestos.com.pe/repuestos-camiones-volvo/fm-fmx/direccion.html"]

In [4]:
# Calibrar opciones y el service necesario
options = Options()
options = webdriver.ChromeOptions()
options.add_argument("start-maximized")
options.add_argument("disable-infobars")
options.add_argument("--disable-extensions")
service = Service(driver_path)

In [5]:
codigo_productos = []
descripciones = []
precios_base = []

In [6]:
output_dir = r"C:\Users\Jesùs\Desktop\Images-Volvo"
# Crear el directorio de salida si no existe
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [7]:
%%time
for web in webs:
    y = 1  # Empezar desde la página 1

    # Iterar para cada web
    url = webdriver.Chrome(service=service, options=options)
    url.get(web)
    
    while True:
        x = 1  # Empezar la iteración desde 1 en cada página       
        while True:
            try:
                # Extraer información
                label_name_info = url.find_element(By.XPATH, f'//*[@id="layer-product-list"]/div[2]/ol/li[{x}]/div/div[3]/strong/a')
                nombre_producto = label_name_info.text        
                
                label_cod_inf = url.find_element(By.XPATH, f'//*[@id="layer-product-list"]/div[2]/ol/li[{x}]/div/div[3]/div[1]/p/span[3]')
                codigo_producto = label_cod_inf.text
                codigo_productos.append(codigo_producto)

                
                label_des_inf = url.find_element(By.XPATH, f'//*[@id="layer-product-list"]/div[2]/ol/li[{x}]/div/div[3]/div[1]/p/span[2]')
                descripcion = label_des_inf.text
                descripciones.append(descripcion)
                
                label_base = url.find_element(By.XPATH, f'//*[@id="layer-product-list"]/div[2]/ol/li[{x}]/div/div[3]/div[2]/div[2]')
                precio_base = label_base.text
                precios_base.append(precio_base)
                
                # Descarga de imágenes
                image_element = url.find_element(By.XPATH, f'//*[@id="layer-product-list"]/div[2]/ol/li[{x}]/div/div[1]/a/img')
                image_url = image_element.get_attribute('src')
                image_data = requests.get(image_url).content
                output_path = os.path.join(output_dir, f'{nombre_producto}-{codigo_producto}.jpg')
                with open(output_path, 'wb') as file:
                    file.write(image_data)

                # Declarando nombre de categoría y subcategoría
                pag = web.split('/')
                # Obtener las dos últimas partes y eliminar la extensión '.html' de la última parte
                pag = pag[-2] + '/' + pag[-1].replace('.html', '')
                
                print(f'Imagen {x} descargada de la pag ({pag}) - {y} y guardada en {os.path.abspath(output_path)}')
                x += 1
            except NoSuchElementException:
                print(f'No hay más artículos en la página. Pasando a la siguiente...\n')
                break
        try:
            next_button = url.find_element(By.XPATH, '//*[@id="layer-product-list"]/div[3]/div[3]/ul/li/a[@class="action  next"]')
            next_button.click()
            time.sleep(1)  # Esperar un momento para que la nueva página cargue completamente
            y += 1
        except NoSuchElementException:
            print("===================================")
            print("Extracción terminada.")
            print("===================================\n")
            url.quit()
            break

Imagen 1 descargada de la pag (fh/cabina) - 1 y guardada en C:\Users\Jesùs\Desktop\Images-Volvo\Amortiguador De Cabina-VO 21171973.jpg
Imagen 2 descargada de la pag (fh/cabina) - 1 y guardada en C:\Users\Jesùs\Desktop\Images-Volvo\Faro Posterior Combinado, Izquierdo-VO 23522307.jpg
Imagen 3 descargada de la pag (fh/cabina) - 1 y guardada en C:\Users\Jesùs\Desktop\Images-Volvo\Faro Posterior Combinado, Derecho-VO 23522308.jpg
Imagen 4 descargada de la pag (fh/cabina) - 1 y guardada en C:\Users\Jesùs\Desktop\Images-Volvo\Guardabarros-VO 21094391.jpg
Imagen 5 descargada de la pag (fh/cabina) - 1 y guardada en C:\Users\Jesùs\Desktop\Images-Volvo\Amortiguador-VO 23111320.jpg
Imagen 6 descargada de la pag (fh/cabina) - 1 y guardada en C:\Users\Jesùs\Desktop\Images-Volvo\Estribo-VO 3175927.jpg
Imagen 7 descargada de la pag (fh/cabina) - 1 y guardada en C:\Users\Jesùs\Desktop\Images-Volvo\Compresor De Aire Acondicionado-VO 84094705.jpg
Imagen 8 descargada de la pag (fh/cabina) - 1 y guardada e

In [8]:
# Crear un DataFrame con los datos extraídos
df = pd.DataFrame({
    'CÓDIGO DE PRODUCTO': codigo_productos,
    'DESCRIPCIÓN': descripciones,
    'PRECIO BASE': precios_base,
})

In [9]:
df

,CÓDIGO DE PRODUCTO,DESCRIPCIÓN,PRECIO BASE
0,VO 21171973,Camiones Volvo,$ 346.58\nPrecio especial\n$ 225.27
1,VO 23522307,Camiones Volvo,$ 667.88
2,VO 23522308,Camiones Volvo,$ 667.88
3,VO 21094391,Camiones Volvo,$ 237.96
4,VO 23111320,Camiones Volvo,$ 261.37\nPrecio especial\n$ 169.90
5,VO 3175927,Camiones Volvo,$ 469.86
6,VO 84094705,Camiones Volvo,"$ 1,531.60"
7,VO 21539730,Camiones Volvo,$ 129.88
8,VO 21111925,Camiones Volvo,$ 261.37\nPrecio especial\n$ 169.90
9,VO 22128971,Camiones Volvo,$ 219.90\nPrecio especial\n$ 142.93


In [10]:
def limpiar_precios(df, col_base='PRECIO BASE', col_descuento='PRECIO CON DESCUENTO'):
    # Inicializar la columna de descuento con 0
    df[col_descuento] = 0.0
    
    # Función para procesar cada fila
    def procesar_precio(precio):
        if '\nPrecio especial\n' in precio:
            precios = re.findall(r'\d+[\d,.]*', precio)
            return float(precios[0].replace(',', '')), float(precios[1].replace(',', ''))
        else:
            return float(precio.replace('$', '').replace(',', '').strip()), 0.0
    
    # Aplicar la función de procesamiento
    df[[col_base, col_descuento]] = df[col_base].apply(lambda x: pd.Series(procesar_precio(x)))
    
    return df

In [11]:
df = limpiar_precios(df, col_base='PRECIO BASE', col_descuento='PRECIO CON DESCUENTO')

In [12]:
df

,CÓDIGO DE PRODUCTO,DESCRIPCIÓN,PRECIO BASE,PRECIO CON DESCUENTO
0,VO 21171973,Camiones Volvo,346.58,225.27
1,VO 23522307,Camiones Volvo,667.88,0.00
2,VO 23522308,Camiones Volvo,667.88,0.00
3,VO 21094391,Camiones Volvo,237.96,0.00
4,VO 23111320,Camiones Volvo,261.37,169.90
5,VO 3175927,Camiones Volvo,469.86,0.00
6,VO 84094705,Camiones Volvo,1531.60,0.00
7,VO 21539730,Camiones Volvo,129.88,0.00
8,VO 21111925,Camiones Volvo,261.37,169.90
9,VO 22128971,Camiones Volvo,219.90,142.93


In [13]:
%%time
# CARGA A SQL
try:
    # Crear la conexión a SQL Server con SQLAlchemy
    connection_string = (
        "mssql+pyodbc://@localhost/"  # En vez de "localhost" cambiarlo por tu servidor
        "TSQL?" # En vez de "TSQL" cambiarlo por tu Base de Datos
        "driver=ODBC+Driver+17+for+SQL+Server"
    )
    # Crear la conexión a SQL Server con SQLAlchemy
    engine = create_engine(connection_string, echo=True)
    # Nombre de la tabla y esquema en SQL Server
    nombre_tabla = 'VOLVO'
    # Cargar los datos en la tabla de SQL Server
    print(df.to_sql(name=nombre_tabla, con=engine, index=False, if_exists='replace'))
    print(f'Data cargada exitosamente en la tabla "{nombre_tabla}"')
except Exception as e:
    print(f'Error: {e}')
finally:
    # No es necesario cerrar la conexión en este caso ya que SQLAlchemy maneja la conexión automáticamente
    pass

2024-05-28 12:38:44,679 INFO sqlalchemy.engine.Engine SELECT CAST(SERVERPROPERTY('ProductVersion') AS VARCHAR)
2024-05-28 12:38:44,679 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-05-28 12:38:44,686 INFO sqlalchemy.engine.Engine SELECT schema_name()
2024-05-28 12:38:44,686 INFO sqlalchemy.engine.Engine [generated in 0.00124s] ()
2024-05-28 12:38:44,710 INFO sqlalchemy.engine.Engine SELECT CAST('test max support' AS NVARCHAR(max))
2024-05-28 12:38:44,710 INFO sqlalchemy.engine.Engine [generated in 0.00122s] ()
2024-05-28 12:38:44,710 INFO sqlalchemy.engine.Engine SELECT 1 FROM fn_listextendedproperty(default, default, default, default, default, default, default)
2024-05-28 12:38:44,719 INFO sqlalchemy.engine.Engine [generated in 0.00138s] ()
2024-05-28 12:38:44,899 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-05-28 12:38:44,908 INFO sqlalchemy.engine.Engine SELECT [INFORMATION_SCHEMA].[TABLES].[TABLE_NAME] 
FROM [INFORMATION_SCHEMA].[TABLES] 
WHERE ([INFORMATION_SCHEMA].[TABLE